In [17]:
import httpx
import json
from typing import Dict
from urllib.parse import quote


In [18]:
def scrape_post(ig_doc_id ,url_or_shortcode: str) -> Dict:
    """Scrape single Instagram post data"""
    print(f"Scraping: {url_or_shortcode}, id: {ig_doc_id}")
    
    if "http" in url_or_shortcode:
        shortcode = url_or_shortcode.split("/p/")[-1].split("/")[0]
    else:
        shortcode = url_or_shortcode

    variables = quote(json.dumps({
        'shortcode':shortcode,'fetch_tagged_user_count':None,
        'hoisted_comment_id':None,'hoisted_reply_id':None
    }, separators=(',', ':')))
    body = f"variables={variables}&doc_id={ig_doc_id}"
    url = "https://www.instagram.com/graphql/query"

    result = httpx.post(
        url=url,
        headers={"content-type": "application/x-www-form-urlencoded"},
        data=body
    )
    data = json.loads(result.content)
    
    print(data)
    
    # return data["data"]["xdt_shortcode_media"]
    return data

In [ ]:
# document id for content query:
#more post content: 
# IG_DOC_ID = "9195084877213487"
#more content
# IG_DOC_ID = "9507655305965624"



#relationships/related : 9480525715305126
# IG_DOC_ID = "9480525715305126"
#acc metadata: 9109150515847101
# IG_DOC_ID = "9109150515847101"
#reels: 9318096484942317
# IG_DOC_ID = "9318096484942317"

In [19]:
 ! pip install pytest-playwright
 ! playwright install

zsh:1: command not found: pip


In [20]:
import asyncio
from playwright.async_api import async_playwright
import re
import json

In [21]:
#This is a dynamic environment that stays open after execution

# Global references for easy access across notebook cells
playwright = await async_playwright().start()
device = playwright.devices["Pixel 5"]
browser = await playwright.chromium.launch(headless=False)  # Browser will stay open
context = await browser.new_context(**device)
page = await context.new_page()
# You now have persistent global references:
# playwright, browser, context, page

In [7]:
! pip install python-dotenv

zsh:1: command not found: pip


In [22]:
from dotenv import load_dotenv
#load the .env file
# get USERNAME and PASSWORD
load_dotenv(override=True)
import os
username = os.getenv("USERNAME")
password = os.getenv("PASSWORD")

In [23]:
username

'databot363792934'

In [24]:
#signon doesn't work rn

import asyncio
from playwright.async_api import TimeoutError

async def signon(page, username, password):
    try:
        await page.goto("https://www.instagram.com/accounts/login/", wait_until="networkidle")

        
        await page.wait_for_selector('input[name="username"]', timeout=10000)
        await asyncio.sleep(0.5)
        await page.fill('input[name="username"]', username)
        await asyncio.sleep(0.5)
        await page.fill('input[name="password"]', password)
        await asyncio.sleep(0.5)

        await page.click('button[type="submit"]')
        await asyncio.sleep(1)

        # Wait for navigation or check for login errors
        await page.wait_for_selector('nav', timeout=10000)  # Wait for main navigation bar as successful login indicator

        # Verify successful login by checking URL or user-specific elements
        if "login" in page.url:
            raise ValueError("Login failed: Incorrect username or password.")

        print("✅ Successfully logged in")
        
        return

    except TimeoutError:
        raise TimeoutError("Login timed out. Check credentials or network connection.")
    
    

In [25]:
await signon(page, username, password)

ValueError: Login failed: Incorrect username or password.

In [46]:
def inspect_post_structure(data, post_id=None):
    """
    Helper function to inspect the structure of an Instagram post JSON
    
    Args:
        data (dict): The raw data dictionary from your scraper
        post_id (str, optional): Specific post ID to inspect. If None, inspects the first post.
    
    Returns:
        None, prints structure information
    """
    # If no specific post_id, use the first one
    if post_id is None and data:
        post_id = next(iter(data.keys()))
    
    if post_id not in data:
        print(f"Post ID {post_id} not found in data")
        return
    
    post_data = data[post_id]
    items = post_data.get("items", [])
    
    print(f"Inspecting Post ID: {post_id}")
    print(f"Number of items: {len(items)}")
    
    if not items:
        print("No items found in this post")
        return
    
    item = items[0]  # Analyze first item
    
    # Print top-level keys
    print("\nTop-level keys:")
    for key in item.keys():
        print(f"- {key}")
    
    # Look specifically for music-related keys
    music_related_keys = [
        key for key in item.keys() 
        if any(music_term in key.lower() for music_term in ["music", "audio", "sound", "clips"])
    ]
    
    print("\nMusic-related keys found:")
    for key in music_related_keys:
        print(f"- {key}")
        
        # If it's a dictionary, show its structure
        if isinstance(item[key], dict):
            print(f"  Subkeys: {list(item[key].keys())}")
        
        # If it's a list, show length and first item structure
        elif isinstance(item[key], list) and item[key]:
            print(f"  List with {len(item[key])} items")
            if isinstance(item[key][0], dict):
                print(f"  First item keys: {list(item[key][0].keys())}")


In [47]:
import asyncio
import re
import json
import os
from urllib.request import urlretrieve

async def scrape_profile_posts(profile_url: str, page, max_posts=None):
    endpoint_pattern = re.compile(r"/api/v1/media/\d+/info/")
    intercepted_posts = {}
    clicked_hrefs = set()

    async def intercept_response(response):
        if endpoint_pattern.search(response.url) and response.status == 200:
            data = await response.json()
            media_id = data.get('items', [{}])[0].get('id', 'unknown')
            intercepted_posts[media_id] = data
            print(f"✅ Intercepted post {media_id}")

    page.on("response", intercept_response)

    print(f"Navigating to profile: {profile_url}")
    await page.goto(profile_url, wait_until="networkidle")
    await asyncio.sleep(1)  # Give extra time for loading

    # Scroll to load posts in batches, collecting links after each scroll
    last_height = 0
    total_found_posts = 0
    
    while True:
        # Find all current posts
        posts = await page.query_selector_all("div._aagu")
        
        # Collect new post URLs
        for post in posts:
            parent = await post.evaluate_handle("node => node.closest('a')")
            if parent:
                href = await parent.get_attribute('href')
                if href and '/p/' in href and href not in clicked_hrefs:
                    clicked_hrefs.add(href)
                    total_found_posts += 1
                    print(f"🟢 Found post URL ({total_found_posts}): {href}")
        
        # Check if we've reached max_posts
        if max_posts and total_found_posts >= max_posts:
            print(f"Reached target of {max_posts} posts. Stopping scroll.")
            break
            
        # Scroll down
        await page.evaluate("window.scrollTo(0, document.body.scrollHeight);")
        await asyncio.sleep(1)  # Wait for new content to load
        
        # Check if we've reached the bottom (no new posts loaded)
        new_height = await page.evaluate("() => document.body.scrollHeight")
        if new_height == last_height:
            print("Reached bottom of page. No more posts to load.")
            break
        last_height = new_height
        
        print(f"Scrolled, found {total_found_posts} posts so far...")

    print(f"✅ Finished scrolling. Found {len(clicked_hrefs)} post URLs.")

    # Limit posts to process if max_posts is specified
    posts_to_process = list(clicked_hrefs)
    if max_posts and len(posts_to_process) > max_posts:
        posts_to_process = posts_to_process[:max_posts]
    
    # Process each post URL
    for i, href in enumerate(posts_to_process):
        post_url = f"https://www.instagram.com{href}"
        print(f"\n🔗 Processing post {i+1}/{len(posts_to_process)}: {post_url}")
        
        future_response = asyncio.Future()

        async def response_listener(response):
            if endpoint_pattern.search(response.url) and response.status == 200:
                try:
                    data = await response.json()
                    media_id = data.get('items', [{}])[0].get('id', 'unknown')
                    intercepted_posts[media_id] = data
                    print(f"✅ Intercepted post {media_id}")
                    if not future_response.done():
                        future_response.set_result(True)
                except Exception as e:
                    print(f"Error processing response: {e}")

        page.on("response", response_listener)

        try:
            await page.goto(post_url, wait_until="networkidle")
            await asyncio.wait_for(future_response, timeout=10)
        except asyncio.TimeoutError:
            print(f"❌ Failed to intercept data for {post_url}")
        except Exception as e:
            print(f"Error navigating to {post_url}: {e}")

        # Remove listener after interception
        page.remove_listener("response", response_listener)

        await asyncio.sleep(0.5)  # Adjust if needed

    print("Completed scraping posts.")
    return intercepted_posts

async def process_and_save_data(data, output_folder="instagram_data"):
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    
    reduced_json_list = []

    # Process each media item
    for media_id, json_data in data.items():
        items = json_data.get("items", [])
        
        for item in items:
            music_data = item.get("music_metadata")
            
            # Only process posts that have music attached and not unlicensed
            if music_data and not music_data.get("is_explicitly_unlicensed", False):
                # Extract relevant fields
                post_id = item.get("pk")
                reduced_data = {
                    "id": post_id,
                    "timestamp": item.get("taken_at"),
                    "music_title": music_data.get("title"),
                    "music_artist": music_data.get("display_artist"),
                    "image_url": None,
                    "image_path": None
                }

                # Extract first image URL
                image_candidates = item.get("image_versions2", {}).get("candidates", [])
                if image_candidates:
                    image_url = image_candidates[0].get("url")
                    reduced_data["image_url"] = image_url
                    
                    # Download and save the image
                    if image_url:
                        image_path = os.path.join(output_folder, f"{post_id}.jpg")
                        try:
                            urlretrieve(image_url, image_path)
                            reduced_data["image_path"] = image_path
                            print(f"✅ Saved image for post {post_id}")
                        except Exception as e:
                            print(f"❌ Failed to save image for post {post_id}: {e}")

                # Append the post data to the reduced list
                reduced_json_list.append(reduced_data)

    # Save all processed data to a JSON file
    output_json_path = os.path.join(output_folder, "processed_data.json")
    with open(output_json_path, 'w') as f:
        json.dump(reduced_json_list, f, indent=4)
    
    print(f"✅ Saved processed data to {output_json_path}")
    return reduced_json_list

# Example usage (to be run in IPython notebook)
async def run_scraper(profile_url, browser_page, max_posts=None):
    # Collect posts data
    data = await scrape_profile_posts(profile_url, browser_page, max_posts)

    inspect_post_structure(data)
    
    # Process and save the data
    processed_data = await process_and_save_data(data)

    for item in processed_data[:5]:
        print(f"Music: {item['music_title']} by {item['music_artist']}")
    
    # Show summary
    print(f"\nSummary: Processed {len(processed_data)} posts with music")
    return processed_data

# This is just an example of how to use the above functions
profile_url = "https://www.instagram.com/daisychoiii/"
processed_data = await run_scraper(profile_url, page, max_posts=50)

Navigating to profile: https://www.instagram.com/daisychoiii/
🟢 Found post URL (1): /daisychoiii/p/DGkOdP-R9fY/
🟢 Found post URL (2): /daisychoiii/p/DFzJ4LQxlTw/
🟢 Found post URL (3): /daisychoiii/p/DE34artRKWu/
🟢 Found post URL (4): /daisychoiii/p/DC2lg1Bxl0l/
🟢 Found post URL (5): /daisychoiii/p/DCn40cwRXlk/
🟢 Found post URL (6): /daisychoiii/p/DCkm1ECyfpM/
🟢 Found post URL (7): /daisychoiii/p/DCgGK3TxJPY/
🟢 Found post URL (8): /daisychoiii/p/DCgF-AeR5zt/
🟢 Found post URL (9): /daisychoiii/p/DCYuI0YAzB_/
🟢 Found post URL (10): /daisychoiii/p/DA5XY-NxrOX/
🟢 Found post URL (11): /daisychoiii/p/DApb0t6xiq6/
🟢 Found post URL (12): /daisychoiii/p/DAfVk7Ex9MY/
🟢 Found post URL (13): /daisychoiii/p/DADABUHR6EC/
🟢 Found post URL (14): /daisychoiii/p/C9_1WhXx7Ai/
🟢 Found post URL (15): /daisychoiii/p/C9yqOAGxIuB/
🟢 Found post URL (16): /daisychoiii/p/C9rOP38APgi/
🟢 Found post URL (17): /daisychoiii/p/C9jU_gqx4cO/
🟢 Found post URL (18): /daisychoiii/p/C9ATcCThz0g/
🟢 Found post URL (19): /daisy

In [48]:
# Print a summary of first few items
for item in processed_data[:30]:  # First 5 items
    print(f"Post ID: {item['id']}")
    print(f"Music: {item['music_title']} by {item['music_artist']}")
    print(f"Image saved at: {item['image_path']}")
    print("-" * 40)

Post ID: 3058910934110574441
Music: None by None
Image saved at: None
----------------------------------------
Post ID: 3505097182242536012
Music: None by None
Image saved at: None
----------------------------------------
Post ID: 3286384313518434717
Music: None by None
Image saved at: None
----------------------------------------
Post ID: 3171197755602817083
Music: None by None
Image saved at: None
----------------------------------------
Post ID: 3501751626413322367
Music: None by None
Image saved at: None
----------------------------------------
Post ID: 3157667261627489431
Music: None by None
Image saved at: None
----------------------------------------
Post ID: 3144199827326694954
Music: None by None
Image saved at: None
----------------------------------------
Post ID: 3091918915928764380
Music: None by None
Image saved at: None
----------------------------------------
Post ID: 3303731842412984282
Music: None by None
Image saved at: None
----------------------------------------
P

In [38]:
data.keys()

dict_keys(['2832904091686683179_2204228646', '2749203838168565223_2204228646', '2696590266838482408_2204228646', '2576633603258395844_2204228646', '2734087921575442713_2204228646', '2650512754131955009_2204228646', '2571513551639185618_2204228646', '2382427573441098476_2204228646', '2536743637099682362_2204228646', '2636134634929209333_2204228646'])

In [28]:
data['2832904091686683179_2204228646']

{'num_results': 1,
 'more_available': False,
 'items': [{'taken_at': 1651928505,
   'pk': '2832904091686683179',
   'id': '2832904091686683179_2204228646',
   'fbid': '17930513051323068',
   'device_timestamp': 1651928460500624,
   'caption_is_edited': True,
   'strong_id__': '2832904091686683179_2204228646',
   'deleted_reason': 0,
   'has_shared_to_fb': 0,
   'has_delayed_metadata': False,
   'mezql_token': '',
   'share_count_disabled': False,
   'should_request_ads': False,
   'is_reshare_of_text_post_app_media_in_ig': False,
   'is_visual_reply_commenter_notice_enabled': True,
   'like_and_view_counts_disabled': False,
   'is_post_live_clips_media': False,
   'can_modify_carousel': False,
   'is_quiet_post': False,
   'client_cache_key': 'MjgzMjkwNDA5MTY4NjY4MzE3OQ==.3',
   'integrity_review_decision': 'approved',
   'has_privately_liked': False,
   'filter_type': 0,
   'usertags': {'in': [{'position': [0.902991453, 0.9836601298000001],
      'user': {'pk': '26445976593',
       '